In [1]:
%load_ext autoreload

In [10]:
%autoreload

import sys
import os
import json

# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" )

print( sys.path )

import lib.util as du

path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import lib.util_pandas      as up
import lib.util_stopwatch   as sw
import lib.util_code_runner as ucr

from agent_calendaring import CalendaringAgent

import pandas as pd
import openai

debug = True
verbose = True


[/var/genie-in-the-box/src/lib] already in sys.path
['', '/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/site-packages', '/usr/local/lib/python310.zip', '/var/genie-in-the-box/src', '/var/genie-in-the-box/src/lib', '/var/genie-in-the-box/src/notebooks']
Path [/var/genie-in-the-box/src] already in sys.path


In [9]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def ask_chat_gpt_text( preamble, query, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are chatGPT, a helpful chatbot"
question = "What's your name?"

ask_chat_gpt_text( preamble, question, model=GPT_4 )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,692 ms


"My name is ChatGPT. I'm an artificial intelligence designed to assist with information and conversation."

In [21]:
from solution_snapshot_mgr import SolutionSnapshotManager
from agent_refactoring     import RefactoringAgent

In [23]:
%autoreload

path_to_snapshots = du.get_project_root() + "/src/conf/long-term-memory/solutions/"
snapshot_mgr      = SolutionSnapshotManager( path_to_snapshots, debug=False )
exemplar_snapshot = snapshot_mgr.get_snapshots_by_question( "What concerts do I have this week?" )[ 0 ][ 1 ]
similar_snapshots = snapshot_mgr.get_snapshots_by_code_similarity( exemplar_snapshot, threshold=90.0 )

agent         = RefactoringAgent( similar_snapshots=similar_snapshots, path_to_solutions="/src/conf/long-term-memory/solutions", debug=True, verbose=True )
response_dict = agent.run_prompt()


------------------------------------------------------------------------------------------------------------------------
- Found [32] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [when was juan born] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapshot.question [what day is today]
Synonymous question [whats todays day and date] for snapshot.question [what day is today]
Synonymous question [what is the day and date for today] for snapshot.question [what day is today]
Synonymous question [whats the date] for snapshot.question [what day is today]
Synonymous question [say what day is today] for snapshot.question [what day is today]
Synonymous question [hey

In [22]:
%autoreload
agent.update_code( debug=False )

AttributeError: 'CalendaringAgent' object has no attribute 'update_code'

In [ ]:
%autoreload
agent.update_examples( debug=True )

### Write the freshly minted function to both the repo and the Io directories, and create all the meditator necessary to execute the code

In [74]:
import os

def write_code_to_unique_files( lines, agent_src_root, agent_lib_chunk, file_name_prefix, suffix=".py" ):
    
    # Get the list of files in the agent_lib_path directory
    files = os.listdir( agent_src_root + agent_lib_chunk )
    
    # Count the number of files with the name {file_name_prefix}{}{suffix}"
    count = sum( 1 for file in files if file.startswith( file_name_prefix ) and file.endswith( suffix ) )
    
    # Format the file name with the count
    file_name = f"{file_name_prefix}{count}{suffix}"
    util_name = f"{file_name_prefix}{count}" 
    
    # Write the file to the repo path
    repo_path = os.path.join( agent_src_root, agent_lib_chunk, file_name )
    print( f"Writing file [{repo_path}]... ", end="" )
    du.write_lines_to_file( repo_path, lines )
    # Set the permissions of the file to be world-readable and writable
    os.chmod( repo_path, 0o666 )
    print( "Done!" )
    
    # Write the file to the io/execution path
    io_path = f"{du.get_project_root()}/io/{agent_lib_chunk}{file_name}"
    print( f"Writing file [{io_path}]... ", end="" )
    du.write_lines_to_file( io_path, lines )
    # Set the permissions of the file to be world-readable and writable
    os.chmod( io_path, 0o666 )
    print( "Done!", end="\n\n" )
    
    # Build import 'as' string:
    non_digits   = "util_calendaring_2".split( "_" )[ :-1 ]
    first_digits = "".join( [ item[ 0 ] for item in non_digits ] )
    abbrev       = f"{first_digits}{count}"
    as_chunk     = f"as {abbrev}"
    import_str   = f"import {agent_lib_chunk.replace( '/', '.' )}{util_name} {as_chunk}"
    
    results = { 
        "file_name": file_name,
        "repo_path": repo_path,
          "io_path": io_path,
            "count": count,
           "abbrev": abbrev,
           "import": import_str
    }

    return results

# Usage example
agent_src_root  = du.get_project_root() + "/src/" 
agent_lib_chunk = "lib/autogen/"
foo = [ "import os", "import json" ]

code_write_metadata = write_code_to_unique_files( response_dict[ "code" ], agent_src_root, agent_lib_chunk, "util_calendaring_", suffix=".py" )
# 
print( f"File     count: [{code_write_metadata[ 'count' ]}]" )
print( f"File file_name: [{code_write_metadata[ 'file_name' ]}]" )
print( f"File repo_path: [{code_write_metadata[ 'repo_path' ]}]" )
print( f"File   io_path: [{code_write_metadata[ 'io_path' ]}]" )
print( f"File    abbrev: [{code_write_metadata[ 'abbrev' ]}]" )
print( f"File    import: [{code_write_metadata[ 'import' ]}]" )

Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_4.py]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_4.py]... Done!

File     count: [4]
File file_name: [util_calendaring_4.py]
File repo_path: [/var/genie-in-the-box/src/lib/autogen/util_calendaring_4.py]
File   io_path: [/var/genie-in-the-box/io/lib/autogen/util_calendaring_4.py]
File    abbrev: [uc4]
File    import: [import lib.autogen.util_calendaring_4 as uc4]


## Update the examples dictionary so that it contains a list of source code needed to execute the freshly minted function

In [80]:
def update_example_code( refactoring_response_dict, code_metadata, debug=False ):
    
    function_name = refactoring_response_dict[ "function_name" ]
    
    # Update the examples dictionary so that it contains a list of source code needed to execute the freshly minted function
    for key, value in refactoring_response_dict[ "examples" ].items():
        
        if debug: print( f"Before: {key}: {value}" )
        if type( value ) is not list:
            value = value.replace( function_name, f"{code_metadata[ 'abbrev' ]}.{function_name}" )
            value = [ code_metadata[ 'import' ], value ]
        else:
            print( f"No need to update [{value}]" )
        if debug: print( f" After: {key}: {value}" )
        
        refactoring_response_dict[ "examples" ][ key ] = value
        
    return refactoring_response_dict

response_dict = update_example_code( response_dict.copy(), code_write_metadata )
response_dict
    

No need to update [['import lib.autogen.util_calendaring_3 as uc3', "print(uc3.get_events_by_week_and_type(df, 'Concert').to_json(orient='records', lines=True))"]]
No need to update [['import lib.autogen.util_calendaring_3 as uc3', "print(uc3.get_events_by_week_and_type(df, 'Birthday').to_json(orient='records', lines=True))"]]


{'thoughts': 'The two code snippets are very similar. They both filter a DataFrame based on the event type and the date range of the current week. The only difference is the event type they are filtering for. Therefore, we can create a function that takes the event type as a parameter and filters the DataFrame accordingly. We will also need to handle the case where the DataFrame is empty after filtering, in which case we will return an empty DataFrame.',
 'code': ['import pandas as pd',
  'from datetime import datetime, timedelta',
  'def get_events_by_week_and_type(df, event_type):',
  "    df['start_date'] = pd.to_datetime(df['start_date'])",
  "    df['end_date'] = pd.to_datetime(df['end_date'])",
  '    now = pd.Timestamp.now()',
  "    start_of_week = now - pd.to_timedelta(now.dayofweek, unit='d')",
  "    end_of_week = start_of_week + pd.to_timedelta(6, unit='d')",
  "    event_mask = df['event_type'] == event_type",
  "    date_mask = (df['start_date'] <= end_of_week) & (df['end

In [82]:
response_dict[ "examples" ]

{'what concerts do i have this week': ['import lib.autogen.util_calendaring_3 as uc3',
  "print(uc3.get_events_by_week_and_type(df, 'Concert').to_json(orient='records', lines=True))"],
 'what birthdays do i have this week': ['import lib.autogen.util_calendaring_3 as uc3',
  "print(uc3.get_events_by_week_and_type(df, 'Birthday').to_json(orient='records', lines=True))"]}

In [101]:
import solution_snapshot as ss

In [102]:
def update_snapshot_code( snapshots, refactoring_response_dict, debug=False ):
    
    for snapshot in snapshots:
        
        if debug:
            du.print_banner( f"BEFORE updating `{snapshot[ 1 ].question}`...", prepend_nl=False )
            for line in snapshot[ 1 ].code: print( line )
            print( "\n" )
            
        # Update the code, using the question as the key
        new_code = refactoring_response_dict[ "examples" ][ snapshot[ 1 ].question ]
        snapshot[ 1 ].code           = new_code
        snapshot[ 1 ].code_embedding = ss.SolutionSnapshot.generate_embedding( " ".join( new_code ) )
        snapshot[ 1 ].write_to_file()
        
        if debug:
            du.print_banner( f" AFTER updating `{snapshot[ 1 ].question}`...", prepend_nl=False )
            for line in snapshot[ 1 ].code: print( line )
            print( "\n" )
        
    
update_snapshot_code( similar_snapshots, response_dict, debug=True )

------------------------------------------------------------------------------------------------------------------------
- BEFORE updating `what concerts do i have this week`...
------------------------------------------------------------------------------------------------------------------------

import lib.autogen.util_calendaring_3 as uc3
print(uc3.get_events_by_week_and_type(df, 'Concert').to_json(orient='records', lines=True))


Generating embedding for [import lib.autogen.util_calendaring_3 as uc3 print(uc3.get_event...]...
Generating embedding for [import lib.autogen.util_calendaring_3 as uc3 print(uc3.get_event...]... Done! in 322 ms

solution_file value provided: [what-concerts-do-i-have-this-week-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/what-concerts-do-i-have-this-week-0.json
------------------------------------------------------------------------------------------------------------------------
-  AFTER updating `what concerts do i hav